# **'Project 1 Healthcare': Exploratory Data Analisys**

1.1 Importeer librairies <br>

In [2]:
import sqlite3
import pandas as pd 
import numpy as np
import json
import csv
import requests
import seaborn as sns
import matplotlib.pyplot as plt


## Collection

**Data ophalen met een SQLite query:**

**Rest API**

**CSV bestand**

In [14]:
df = pd.read_csv('Data/data_dinsdag.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Data/data_dinsdag.csv'

1.2.1 Check if it worked

In [ ]:
df.head()

In [ ]:
df.tail()

## **Preprocessing**

 **Inzichten preprocessing** <br>

1. De dataset bevat 4096 rijen en 8 kolommen
2. 2 kolommen hebben dtype float64, de overige 6 hebben dtype object
3. De functies .head() en .tail() maken inzichtelijk dat de dataset gevuld is met uitsluitend numerieke waarden
4. Via .info[Non-null Count] wordt duidelijk dat er geen 0 waarden in de rijen staan
5. De functie .describe() geeft een relatief groot aantal NaN's terug

**Conclusie**
1. Er zijn cellen die onjuiste waarden bevatten, deze waarden zijn niet numeriek.

**Vervolgstappen**
1. Onderzoek hoeveel onjuiste waardes er zijn
2. Onderzoek waar de onjuiste waardes zijn 
3. Bepaal wat je met de waardes gaat doen > Als het er minder dan 20 zijn verwijderen anders overleggen. <br>

**Pseudocode**

1. Itereer over kolommen heen
2. Filter de onjuiste waarde
3. Stop deze waarde in een lijst
6. Verwijder

In [ ]:
# 1.3
# Eerst checken of er waarden zijn die missen. 
df.isna().sum()

1.3 Check if data contains errors / defaults

In [ ]:
# 1.3
df.info() # 6 col have dtype object

In [ ]:
# Zoek uit wat voor niet numerieke data er in zit

# Ik was de waarden oplopend sorteren zodat bovenaan de niet numerieke waarden komen te staan. Dit is geen mooie manier maar het eerste waar ik nu op kom.

df.sort_values(by='length', ascending=False)

# 11 x ? voor genetic; 3x  undefined voor length; 2 x ? mass; 1 x ? exercise; 2 x ? alcohol; 1 x ?



In [ ]:
# Itereer over kolommen heen
# Filter de onjuiste waarde
# Verwijder

for c in df.columns: # de loop
    df[c] = pd.to_numeric(df[c], errors = 'coerce') # de code # to list
    
pd.isna(np.nan) # Check of niet numerieke waarde gewijzigd zijn naar NaN, als dat zo is True


In [ ]:
df.info() #dubbelcheck of alle kolommen nu een numeriek Dtype hebben

In [ ]:
df.isna().any(axis=1).sum() # er zijn precies 10 foutieve waarden, op een dataset van 4096 rijen kan het verwijderen van 10 rijen zonder dat dit impact heeft.

In [ ]:
a = df.isna() 
b = a.any()
print(b)

In [ ]:
a = df.isna().any(axis=1) #zichtbaar maken waar de gemiste waardes zijn
df[a]

In [ ]:
df.isna().sum().sort_values(ascending=False) # Waar missen de meeste waarde?

In [ ]:
missing_percentages = df.isna().sum().sort_values(ascending=False) / len(df)

In [ ]:
missing_percentages.plot(kind='barh')

In [ ]:
df= df.dropna() # verwijderen en checken of het gelukt is.

In [ ]:
df.isna().sum()

In [ ]:
# Verder exploreren.

In [ ]:
df.describe()

# Lifespan van -1 als min


## **Data selection**

**Inzichten dataselectie**

1. 

In [ ]:
sns.boxplot(x=df['lifespan'])

In [ ]:
#We zien de outliers ook terug op de boxplot. Dit zegt ons echter niets over het aantal -1 waardes. Dit checken we dus:
df.loc[df['lifespan'] < 0]

In [ ]:
cols = [c for c in df.columns]

fig, axes = plt.subplots(ncols=len(cols))

for column, axis in zip(cols, axes):
        sns.boxplot(data=df[column], ax=axis) 
        axis.set_title(column)
        
plt.tight_layout()
plt.show()

In [ ]:

# Outliers verwijderen

def remove_outliers(df,columns,n_std):
    for col in columns:
        print('Working on column: {}'.format(col))
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
        
    return df

remove_outliers(df,['lifespan'],6)

In [ ]:
plt.figure(figsize=(10,5))
c= df.corr()
sns.heatmap(c,cmap="BrBG",annot=True)
#

1. **Genetic en lifespan**: Zeer hoge correlatie. 
2. **Length en lifespan**: Zeer lage correlatie
3. **Mass en lifespan**: Negatieve correlatie
4. **Exercise en lifespan**: Lage correlatie
5. **Smoking en lifespan**: Negatieve correlatie
6. **Alcohol en lifespan**: Negatieve correlatie
7. **Sugar en lifespan**: Negatieve correlatie

Vervolgstappen <br>
Ik ben benieuwd naar een model waarbij alleen wordt gekeken naar iemands lifestyle (exercise, smoking, alcohol, sugar) <br>
Ik hoor in de klas dat er verdeelde meningen zijn over genetics, dus ik ga ook kijken naar een model waar genetics niet bij zitten <br>
Ik ben ook benieuwd


In [ ]:
df.hist(figsize=(20,10))

In [ ]:
sns.pairplot(df)

In [ ]:
# Vragen
# Welke bevolkingsgroep? grote mensen, veel roken, worden erg oud (gem 80)
# Mogelijke clusters:
# exercise (veel-weinig) (boolean)

In [ ]:
# dendogram clusters bmi

## **Feature extraction**

In [ ]:
#We berekenen de bmi volgens de standaard formule:
col_bmi = (df.mass/(df.length/100)**2)

#We kijken naar de nieuwe kolom


In [ ]:
# Nieuw dataframe creeren door deze kolom toe te voegen aan de originele dataset en vervolgens op te slaan onder nieuwe variabele naam. Lengte en gewicht gaan eruit.

#pseudo code
# creeer een nieuwe variabele
# verwijder de kolommen met een pd functie
# voeg de nieuwe col toe met een pd functie


df_bmi = df.drop(['length', 'mass'], axis=1)
df_bmi['bmi'] = col_bmi
df = df_bmi
df_bmi.head()

#df_bmi = df.drop(['BMI cat'], axis=1)

df_bmi.head()

In [ ]:
# Dataframe lifestyle

In [ ]:
df_lifestyle = df.drop(['length', 'mass', 'genetic'], axis=1)

In [ ]:
df_lifestyle = df[['exercise', 'smoking', 'alcohol', 'sugar', 'lifespan']]

df_lifestyle.head()

In [ ]:
df_not_genetic = df.drop(['genetic'], axis = 1)
df_ng = df_not_genetic

df_ng.head()

In [ ]:
df.to_csv(index=False)
df_lifestyle.to_csv('df_lifestyle.csv', index=False)


In [ ]:
df_not_genetic.to_csv('df_ng.csv',index=False)
df_bmi.to_csv('df_bmi.csv', index=False)

In [ ]:

# Save df as new table
df.to_sql('bmi', con=dbConnection, if_exists='replace', index=False)

# close Connection
dbConnection.close()

In [ ]:

import sqlite3
dbName = 'C:/Users/31641/MakeAIWork/Project/Data/db_final.sq3lite'
tableName = 'rest_api_netifly'

dbConnection = sqlite3.connect(dbName)

df = df.to_sql(tableName,dbName, sqlite3.Connection)
df_lifestyle = df_lifestyle.to_sql(tableName, dbName, sqlite3.Connection)
df_ng.to_sql(tableName, dbName, sqlite3.Connection)

In [ ]:

import sqlite3
dbName = "rest_server/medisch_centrum_randstad/db.sqlite3"
tableName = "rest_api_netlify"

dbConnection = sqlite3.connect(dbName)

df.to_sql(tableName,dbName, sqlite3.Connection)
#df_lifestyle = df_lifestyle.to_sql(tableName, dbName, sqlite3.Connection)
#df_ng.to_sql(tableName, dbName, sqlite3.Connection)

In [ ]:
# Nog interessant om te checken
# nicotine gehalte in bloed op basis van gerookte sigaretten